In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/archive/postings.csv',header=0)

In [3]:
df.shape

(123849, 31)

In [4]:
df.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')

In [5]:
df['work_type'].unique

<bound method Series.unique of 0         FULL_TIME
1         FULL_TIME
2         FULL_TIME
3         FULL_TIME
4         FULL_TIME
            ...    
123844    FULL_TIME
123845    FULL_TIME
123846    FULL_TIME
123847    FULL_TIME
123848    FULL_TIME
Name: work_type, Length: 123849, dtype: object>

In [6]:
#drop unrelated columns
df = df.drop(['job_posting_url','application_url','application_type','closed_time','posting_domain', 'sponsored','compensation_type','fips','med_salary','skills_desc','remote_allowed','applies'], axis=1)

In [7]:
# choosing only full time and contract job postings,drop other types of jobs(part-time, intern, temporary)
df_cleaned = df[(df['work_type'].isin(['FULL_TIME', 'CONTRACT']))]
df_cleaned.shape

(110931, 19)

In [8]:
#stats for missing values of each columnb
missing_values = df_cleaned.isnull().sum()

print(missing_values)

job_id                            0
company_name                   1424
title                             0
description                       7
max_salary                    83161
pay_period                    77949
location                          0
company_id                     1422
views                          1599
min_salary                    83161
formatted_work_type               0
original_listed_time              0
expiry                            0
formatted_experience_level    26591
listed_time                       0
work_type                         0
currency                      77949
normalized_salary             77949
zip_code                      19228
dtype: int64


In [9]:
# Calculate the number of missing values per row
missing_values_per_row = df_cleaned.isnull().sum(axis=1)

# Drop rows with more than 10 missing values (adjust the threshold as needed)
df_cleaned = df_cleaned[missing_values_per_row <= 6]

df_cleaned.shape

(106222, 19)

In [10]:
# drop all rows without company info
df_cleaned = df_cleaned.dropna(subset=['company_name'])

In [11]:
df_cleaned.shape

(105796, 19)

In [12]:
#changing time columns format from u-8
import datetime

# change time format
df_cleaned['original_listed_time'] = pd.to_datetime(df_cleaned['original_listed_time'], unit='ms')
df_cleaned['expiry'] = pd.to_datetime(df_cleaned['expiry'], unit='ms')
df_cleaned['listed_time'] = pd.to_datetime(df_cleaned['listed_time'], unit='ms')

In [13]:
# drop jobs without any salary info. 
df_cleaned = df_cleaned.dropna(subset=['max_salary','pay_period','min_salary','normalized_salary','currency'],how='all')
df_cleaned.shape

(32556, 19)

In [14]:
#df_cleaned.to_csv('cleaned_posting_32556_salary_info.csv', index=False) 

In [15]:
df_cleaned.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,min_salary,formatted_work_type,original_listed_time,expiry,formatted_experience_level,listed_time,work_type,currency,normalized_salary,zip_code
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,17.0,Full-time,2024-04-17 23:45:08,2024-05-17 23:45:08,NaN,2024-04-17 23:45:08,FULL_TIME,USD,38480.0,8540.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,45000.0,Full-time,2024-04-16 14:26:54,2024-05-16 14:26:54,NaN,2024-04-16 14:26:54,FULL_TIME,USD,55000.0,45202.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,140000.0,Full-time,2024-04-12 04:23:32,2024-05-12 04:23:32,NaN,2024-04-12 04:23:32,FULL_TIME,USD,157500.0,11040.0
6,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,YEARLY,United States,81942316.0,7.0,60000.0,Contract,2024-04-11 18:43:39,2024-05-11 18:43:39,NaN,2024-04-11 18:43:39,CONTRACT,USD,180000.0,NaN
16,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,HOURLY,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,2024-04-11 17:14:46,2024-05-11 17:14:46,NaN,2024-04-11 17:14:46,FULL_TIME,USD,52000.0,44102.0


In [16]:
# checking if hourly salary list hourly wage
df_hourly = df_cleaned[(df_cleaned['pay_period'] == 'HOURLY')]
df_hourly['normalized_salary'].describe()

count    1.203600e+04
mean     3.575726e+05
std      8.682579e+06
min      1.456000e+04
25%      4.252560e+04
50%      5.616000e+04
75%      9.152000e+04
max      5.356000e+08
Name: normalized_salary, dtype: float64

In [17]:
print(df_hourly['max_salary'].describe())
print(df_hourly['min_salary'].describe())

count      8498.000000
mean        253.102448
std        5389.509095
min           7.000000
25%          24.602500
50%          33.585000
75%          55.000000
max      275000.000000
Name: max_salary, dtype: float64
count      8498.000000
mean        210.097032
std        4550.735878
min           7.000000
25%          19.500000
50%          25.580000
75%          43.255000
max      240000.000000
Name: min_salary, dtype: float64


In [18]:
print(df_hourly['max_salary']>=2000)

0         False
16        False
45        False
62        False
65        False
          ...  
123819    False
123821    False
123832    False
123837    False
123843    False
Name: max_salary, Length: 12036, dtype: bool


In [19]:
df_yearly = df_cleaned[(df_cleaned['pay_period'] == 'YEARLY')]
df_yearly['normalized_salary'].describe()

count    1.994500e+04
mean     1.207295e+05
std      7.279382e+05
min      0.000000e+00
25%      7.398355e+04
50%      1.025000e+05
75%      1.425000e+05
max      1.025000e+08
Name: normalized_salary, dtype: float64

In [41]:
df_cleaned.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'min_salary',
       'formatted_work_type', 'original_listed_time', 'expiry',
       'formatted_experience_level', 'listed_time', 'work_type', 'currency',
       'normalized_salary', 'zip_code'],
      dtype='object')